In [1]:
from langchain_community.document_loaders import CSVLoader

c:\Users\Shahroz\Desktop\Banking FAQ Chatbot using RAG\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Load FAQ Data using CSVLoader

In [2]:
loader = CSVLoader(file_path='banking_app_faqs.csv')

docs = loader.load()

In [3]:
print(len(docs))
print(docs[0])

15
page_content='question: How can I create a new bank account?
answer: You can create a new bank account by registering through the banking app, completing identity verification, and submitting the required documents.' metadata={'source': 'banking_app_faqs.csv', 'row': 0}


Split Documents into Chunks

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)   

texts = text_splitter.split_documents(docs)
print(len(texts))
print(texts[14])

15
page_content='question: How do I close my bank account?
answer: To close your bank account, submit a closure request through the app or visit your nearest bank branch.' metadata={'source': 'banking_app_faqs.csv', 'row': 14}


Create Embeddings

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
# from langchain.retrievers.document_compressors import LLMChainExtractor

In [6]:
# Initialize Huggingface embedding
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

Store Embeddings in Vector DB

In [7]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

Create Retrievers

In [8]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [9]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:latest")

In [10]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt telling the model how to use retrieved context
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant for a banking app. Use the provided context to answer the user's question. If the answer is not in the context, say you don't know and do not make up facts."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

# Helper: turn list of docs into a single context string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build RAG chain using LCEL (no langchain.chains module needed in v1+)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("How do I transfer money to another bank account?")
print(response)

The answer to your question is the same as in the previous two questions:

You can transfer money by selecting the Fund Transfer option, entering beneficiary details, and confirming the transaction using OTP verification.
